### import

In [ ]:
# --- iPython Config --- #
from IPython import get_ipython
if 'IPython.extensions.autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')
else:
    get_ipython().run_line_magic('reload_ext', 'autoreload')
%autoreload 2

# --- System and Path --- #
import os
import sys
REPO_PATH = os.path.abspath(os.path.join('..'))
if REPO_PATH not in sys.path:
    sys.path.append(REPO_PATH)
import warnings
warnings.filterwarnings("ignore")

# --- Data Manipulation --- #
import pandas as pd
import numpy as np
from tqdm import tqdm # Progress bar

# --- Financial Data --- #
import yfinance as yf

# --- Modules --- #
from src import *

# --- Visualization --- #
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({
    # Axes
    'axes.titlesize': 16,
    'axes.titlepad': 20,
    'axes.labelsize': 12,
    'axes.edgecolor': (0.1, 0.1, 0.1),
    'axes.labelcolor': (0.1, 0.1, 0.1),
    'axes.linewidth': 1,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.bottom': True,
    'axes.spines.left': True,
    'axes.grid': True,

    # Grid
    'grid.alpha': 0.7,
    'grid.linestyle': '--',
    'grid.linewidth': 0.6,

    # Lines
    'lines.linewidth': 1.5,
    'lines.markeredgewidth': 0.0,

    # Scatter plot
    'scatter.marker': 'x',

    # Ticks
    'xtick.labelsize': 12,
    'xtick.color': (0.1, 0.1, 0.1),
    'xtick.direction': 'in',
    'ytick.labelsize': 12,
    'ytick.color': (0.1, 0.1, 0.1),
    'ytick.direction': 'in',

    # Figure output
    'figure.figsize': (10, 6),
    'figure.dpi': 200,
    'savefig.dpi': 200,

    # Text
    'text.color': (0.2, 0.2, 0.2),

    # Font
    "font.family": ["serif",
                    "Tahoma"] # TH Font
})

In [ ]:
# Read Data
df_transactions = pd.read_excel(REPO_PATH+'/data/private/main/'+'Transactions.xlsx') # raw

# preprocessing

In [ ]:
def preprocess_securities(df):
    # Select Features
    df = df[['Date',
             'Position',
             'Ticker',
             'Executed Price (USD)',
             'Shares',
             ]]

    # Normalize Date
    df['Date'] = df['Date'].apply(lambda x: x.normalize())

    # New Features
    # Volume
    df['Position'] = df['Position'].apply(lambda x: 1 if x == "Buy" else -1)
    df['Volume'] = df['Shares'] * df['Position']
    df.drop(columns=['Shares', 'Position'], inplace=True)

    return df

df_transactions = preprocess_securities(df_transactions)

In [ ]:
def stock_split(df, ticker, split_date, split_ratio):
    # NVDA Stock Split 2024-06-10, 10:1

    # Volume Adjustment
    df.loc[
        (df["Ticker"] == ticker) & (df["Date"] < split_date),
        "Volume",
    ] = (df["Volume"] * split_ratio)

    # Price Adjustment
    df.loc[
        (df["Ticker"] == ticker) & (df["Date"] < split_date),
        "Executed Price (USD)",
    ] = (df["Executed Price (USD)"] / split_ratio)
    return df

df_transactions = stock_split(df_transactions, 'NVDA', '2024-06-10', 10)

In [ ]:
def rename_ticker(df):
    # Rename Ticker from BRK.B to BRK-B for yfinance support
    df['Ticker'] = df['Ticker'].apply(lambda x: x.replace('.', '-'))

    return df

df_transactions =  rename_ticker(df_transactions)

# Portfolio

In [ ]:
df_portfolio = df_transactions.copy()

In [ ]:
def aggregate_intraday_to_daily(df):
    df['Date'] = df['Date'].dt.normalize()

    df = (
        df.groupby(["Ticker", "Date"])
        .apply(lambda x: pd.Series({
            "Average Cost Price (USD)": np.average(x["Executed Price (USD)"], weights=x["Volume"]),
            "Volume": x["Volume"].sum()
        }))
        .reset_index()
    )

    return df

df_portfolio = aggregate_intraday_to_daily(df_portfolio)

In [ ]:
def cumulative_volume(df):
    df["Cumulative Volume"] = df.groupby("Ticker")["Volume"].cumsum()

    # remove small values resulting from decimal point arithmetic operations
    least_significant_digit = 1e-6
    df["Cumulative Volume"] = df["Cumulative Volume"].apply(
        lambda x: 0 if abs(x) < least_significant_digit else x
    )

    df.drop(columns=["Volume"], inplace=True)
    return df

df_portfolio = cumulative_volume(df_portfolio)

In [ ]:
def daily_basis(df):
    TODAY = pd.Timestamp.today()
    all_dates = pd.date_range(start=df["Date"].min(), end=TODAY)

    # Ensure all dates are present for each ticker
    # and forward-filling
    df = (
        df.set_index("Date")
        .groupby("Ticker")
        # reindex
        .apply(lambda x: x.reindex(all_dates))
        .drop(columns="Ticker")

        # forward-fill
        .groupby("Ticker")
        .ffill()

        .reset_index()
        .rename(columns={"level_1": "Date"})
    )

    return df

df_portfolio = daily_basis(df_portfolio)

## Market Price

In [ ]:
DataTerminal = DataTerminal()

In [ ]:
tickers = set(df_portfolio['Ticker'].unique())
# tickers.add("THB=X")
print(tickers)

df_yf = DataTerminal.fetch_data(tickers)

In [ ]:
def merge_yf_to_portfolio(df_portfolio, df_yf):
    # Select useful features
    df_yf = df_yf[['Ticker', 'Date', 'Adj Close']]

    # Apply daily basis function
    df_yf = daily_basis(df_yf)

    # Merge dataframes
    df_portfolio = df_portfolio.merge(df_yf, on=['Ticker', 'Date'], how='left')
    df_portfolio.rename(columns={'Adj Close': 'Market Price (USD)'}, inplace=True)

    # Forward fill missing market prices
    df_portfolio['Market Price (USD)'] = df_portfolio.groupby('Ticker')['Market Price (USD)'].ffill()

    return df_portfolio

# Apply the function
df_portfolio = merge_yf_to_portfolio(df_portfolio, df_yf)

In [ ]:
def market_value(df):
    # Asset Value
    df["Asset Value (USD)"] = (
        df["Market Price (USD)"] * df["Cumulative Volume"]
    )

    # Portfolio Value
    portfolio_value = df.groupby("Date")["Asset Value (USD)"].sum()
    portfolio_value.name = "Portfolio Value (USD)"

    df = df.merge(
        portfolio_value, how="left", left_on="Date", right_on="Date"
    )
    return df

df_portfolio = market_value(df_portfolio)

In [ ]:
def unrealized_pnl(df):
    # Unrealized PnL (USD) occurs when the security is still hold
    # Asset Unrealized PnL (USD)
    df["Asset Unrealized PnL (USD)"] = abs(df["Cumulative Volume"]) * (df["Market Price (USD)"] - df["Average Cost Price (USD)"])
    # Asset Unrealized PnL (%)
    df["Asset Unrealized PnL (%)"] = (df["Market Price (USD)"] - df["Average Cost Price (USD)"])*100 / df["Average Cost Price (USD)"]

    # Portfolio Unrealized PnL (USD)
    portfolio_unrealized_pnl = df.groupby("Date")["Asset Unrealized PnL (USD)"].sum()
    portfolio_unrealized_pnl.name = "Portfolio Unrealized PnL (USD)"
    df = df.merge(
        portfolio_unrealized_pnl, how="left", left_on="Date", right_on="Date"
    )
    # Portfolio Unrealized PnL (%)
    df["Portfolio Unrealized PnL (%)"] = df["Portfolio Unrealized PnL (USD)"] / df["Portfolio Value (USD)"] *100

    return df


df_portfolio = unrealized_pnl(df_portfolio)

## Cash - USD Deposit

In [ ]:
filename = '20241110-Deposit.csv'
df_deposit = pd.read_csv(REPO_PATH+'/data/private/csv/'+filename,
                         index_col='Date',
                         parse_dates=True)

In [ ]:
# select features
df_deposit = df_deposit[['Balance', 'Cumulative Deposit (USD)']]
df_deposit.rename(columns={'Balance': 'Cash'}, inplace=True)
df_deposit.reset_index(inplace=True)

In [ ]:
def merge_deposit_to_portfolio(df_portfolio, df_deposit):
    # merge
    df_portfolio = df_portfolio.merge(df_deposit, on='Date', how='left')

    # forward fill
    df_portfolio['Cash'] = df_portfolio.groupby('Ticker')['Cash'].ffill()
    df_portfolio['Cumulative Deposit (USD)'] = df_portfolio.groupby('Ticker')['Cumulative Deposit (USD)'].ffill()

    return df_portfolio

df_portfolio = merge_deposit_to_portfolio(df_portfolio, df_deposit)

In [ ]:
df_portfolio

In [ ]:
def save_to_csv(df, REPO_PATH=REPO_PATH):

    time_now = pd.Timestamp.now().strftime("%Y%m%d")

    filename = time_now + '-Portfolio.csv'
    folderpath = REPO_PATH + '/data/private/csv/'
    filepath = folderpath + filename

    df.to_csv(filepath, index=False)

save_to_csv(df_portfolio)

# Metrics

In [ ]:
df_portfolio_metrics = df_portfolio.copy()

In [ ]:
# aggregate
df_portfolio_metrics = df_portfolio_metrics.groupby('Date')[['Portfolio Value (USD)', 'Portfolio Unrealized PnL (USD)', 'Portfolio Unrealized PnL (%)', 'Cash', 'Cumulative Deposit (USD)']].mean()

In [ ]:
def portfolio_profit(df):
    # Portfolio Net Profit (USD)
    df["Portfolio Net Profit (USD)"] = (df["Portfolio Value (USD)"] + df['Cash']) - df["Cumulative Deposit (USD)"]

    # ROI (%)
    df["ROI (%)"] = df["Portfolio Net Profit (USD)"] / df["Cumulative Deposit (USD)"] * 100

    return df

df_portfolio_metrics = portfolio_profit(df_portfolio_metrics)

## Sharpe Ratio
$$Sharpe\ ratio = \frac{R_p - R_f}{\sigma_p}$$

In [ ]:
def fetch_risk_free_rate(ticker='^TNX'):

    # Risk Free Rate
    # CBOE Interest Rate 10 Year T No
    risk_free_rate = DataTerminal.fetch_data([ticker])  # percent
    # no data on market closing day

    risk_free_rate.drop_duplicates(subset="Date", inplace=True)
    risk_free_rate.set_index("Date", inplace=True)
    risk_free_rate.drop(columns=["Ticker"], inplace=True)
    risk_free_rate.rename(columns={"Adj Close": "Annual Rate (%)"}, inplace=True)

    days_in_year=252
    risk_free_rate['Daily Rate (%)'] = risk_free_rate['Annual Rate (%)'] / days_in_year

    # select useful features
    risk_free_rate = risk_free_rate[['Annual Rate (%)', 'Daily Rate (%)']]

    return risk_free_rate

risk_free_rate = fetch_risk_free_rate()

In [ ]:
def dynamic_sharpe_ratio(df, risk_free_rate):

    df_sharpe = df.groupby('Date')['ROI (%)'].mean().reset_index()
    df_sharpe.set_index('Date', inplace=True)
    df_sharpe['Portfolio Daily Return (%)'] = df_sharpe['ROI (%)'].diff()

    # merge risk free rate
    df_sharpe = df_sharpe.merge(risk_free_rate['Daily Rate (%)'], on='Date', how='left')
    # drop row for Non-Trading Day NaN
    df_sharpe.dropna(subset=['Daily Rate (%)'], inplace=True)

    # excess return
    df_sharpe['Excess Daily Return (%)'] = df_sharpe['Portfolio Daily Return (%)'] - df_sharpe['Daily Rate (%)']

    # Sharpe Ratio
    df_sharpe['Dynamic Sharpe Ratio'] = df_sharpe['Excess Daily Return (%)'].expanding().sum() / (df_sharpe['Excess Daily Return (%)'].expanding().std() * np.sqrt(df_sharpe['Excess Daily Return (%)'].expanding().count()))

    return df_sharpe

df_sharpe = dynamic_sharpe_ratio(df_portfolio_metrics, risk_free_rate)

In [ ]:
def merge_sharpe_to_portfolio(df_portfolio_metric, df_sharpe):
    df_portfolio_metric = df_portfolio_metric.merge(df_sharpe['Dynamic Sharpe Ratio'], on='Date', how='left')
    # forward fill
    df_portfolio_metric['Dynamic Sharpe Ratio'] = df_portfolio_metric['Dynamic Sharpe Ratio'].ffill()

    return df_portfolio_metric

df_portfolio_metrics = merge_sharpe_to_portfolio(df_portfolio_metrics, df_sharpe)

In [ ]:
def dynamic_sortino_ratio(df, risk_free_rate):

    df_sortino = pd.DataFrame(index=df.index)
    df_sortino['return_1d'] = df['ROI (%)'].diff()

    # merge risk free rate
    df_sortino = df_sortino.merge(risk_free_rate['Daily Rate (%)'], on='Date', how='left')
    # drop row for Non-Trading Day NaN
    df_sortino.dropna(subset=['Daily Rate (%)'], inplace=True)

    # excess return
    excess_return = df_sortino['return_1d'] - df_sortino['Daily Rate (%)']
    excess_return[excess_return > 0] = 0

    downside_std = np.sqrt((excess_return**2).expanding().sum() / excess_return.expanding().count())

    df['Dynamic Sortino Ratio'] = (df_sortino['return_1d']-df_sortino['Daily Rate (%)']).expanding().sum() / (downside_std * np.sqrt((df_sortino['return_1d']-df_sortino['Daily Rate (%)']).expanding().count()))
    # forward fill
    df['Dynamic Sortino Ratio'] = df['Dynamic Sortino Ratio'].ffill()

    return df

df_portfolio_metrics = dynamic_sortino_ratio(df_portfolio_metrics, risk_free_rate)

In [ ]:
def return_metrics(df):
    # Mean Return
    df['Mean return (%)'] = df['Portfolio Unrealized PnL (%)'].diff().expanding().mean() *100

    # Standard Deviation
    df['Std return (%)'] = df['Mean return (%)'].expanding().std()

    # Volatility 30 days
    df['Volatility_30d (%)'] = df['Mean return (%)'].rolling(30).std() * np.sqrt(30)


    return df

df_portfolio_metrics = return_metrics(df_portfolio_metrics)

In [ ]:
def cvar(df):
    # CVaR 95%
    return_1d = df['Portfolio Unrealized PnL (%)'].diff()

    cvar_95 = return_1d.expanding().apply(lambda x: x[x < x.quantile(0.05)].mean(), raw=False)

    df['CVaR 95%'] = cvar_95

    return df

df_portfolio_metrics = cvar(df_portfolio_metrics)

In [ ]:
df_portfolio.info()

In [ ]:
df_portfolio_metrics.info()

In [ ]:
# def save_to_csv(df, REPO_PATH=REPO_PATH):
#     df.reset_index(inplace=True)

#     time_now = pd.Timestamp.now().strftime("%Y%m%d")

#     filename = time_now + '-PortfolioMetrics.csv'
#     folderpath = REPO_PATH + '/data/private/csv/'
#     filepath = folderpath + filename

#     df.to_csv(filepath, index=False)

# save_to_csv(df_portfolio_metrics)